In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
%matplotlib inline

In [31]:
reviews = pd.read_csv('./Amsterdam/reviews2.csv')
listings_desc = pd.read_csv('./Amsterdam/listings2.csv')
listings = pd.read_csv('./Amsterdam/listings.csv')
neighbour = pd.read_csv('./Amsterdam/neighbourhoods.csv')
calendar = pd.read_csv('./Amsterdam/calendar.csv')

#C:\Users\avadi\Desktop\DataScience_Udacity\Project 1\Athens

In [41]:
listings_desc.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [58]:
calendar[calendar.price.isnull()].listing_id.unique()





array([          12550549,           32234160,           37591658,
                 37614789, 720266014528319438,           18631508],
      dtype=int64)

In [62]:
calendar.listing_id.unique().shape

(6809,)

In [33]:
calendar[calendar.listing_id == 2818]

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,2818,2022-12-05,f,$59.00,$59.00,3,1125
1,2818,2022-12-06,t,$59.00,$59.00,3,1125
2,2818,2022-12-07,t,$59.00,$59.00,3,1125
3,2818,2022-12-08,f,$59.00,$59.00,3,1125
4,2818,2022-12-09,f,$59.00,$59.00,3,1125
...,...,...,...,...,...,...,...
360,2818,2023-11-30,f,$75.00,$75.00,3,1125
361,2818,2023-12-01,f,$76.00,$76.00,3,1125
362,2818,2023-12-02,f,$76.00,$76.00,3,1125
363,2818,2023-12-03,f,$76.00,$76.00,3,1125
